In [3]:
##!conda install -c anaconda beautifulsoup4 -y
##!conda install -c anaconda lxml -y
##!conda install -c anaconda requests -y
##!conda config --add channels conda-forge
##!conda install -c conda-forge geopy --yes
##!conda install -c conda-forge folium=0.5.0 --yes
##  imports and functions
from bs4 import BeautifulSoup
import requests
import lxml
import csv
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from geopy import distance
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML     
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
RVA_Cities =['richmond, va','chesterfield, va', 'midlothian, va', 'henrico, va', 'glen allen, va', 'ashland, va','chmamberlayne, va', 'mechanicsville, va']
new_job_addr = '7100 Forest Ave 23226'
rva_gs = ['Grocery Store', 'Supermarket','Drugstore', 'Pharmacy', 'Market', 'Coffee Shop', 'Wine Shop','Gym / Fitness Center','Salon / Barbershop','Gym', 'Sports Club', 'Farmers Market', 'Food & Drink Shop', 'Performing Arts Venue','Museum', 'Theater','Art Museum','Dog Run','Social Club','State / Provincial Park', 'Bank', 'Lake']


In [4]:
zipfile = pd.read_csv("https://move-econresearch-prod.s3-us-west-2.amazonaws.com/listings/core/current_month/RDC_Inventory_Core_Metrics_Zip_Current.csv")

In [5]:
zip2 = zipfile[['postal_code','zip_name', 'median_listing_price','active_listing_count','average_listing_price','total_listing_count']]
##zip2.head()
rva_df = zip2.loc[zip2['zip_name'].isin(RVA_Cities)]
rva_df

,postal_code,zip_name,median_listing_price,active_listing_count,average_listing_price,total_listing_count
228,23236,"richmond, va",292034.0,96.0,320525.0,192.0
318,23229,"henrico, va",372300.0,58.0,565452.0,135.0
1692,23228,"henrico, va",223050.0,61.0,213530.0,151.0
2052,23233,"henrico, va",400000.0,71.0,461825.0,145.0
2275,23116,"mechanicsville, va",381040.0,168.0,409562.0,342.0
2524,23238,"henrico, va",379550.0,81.0,538906.0,143.0
3471,23231,"henrico, va",264775.0,99.0,303743.0,227.0
3697,23222,"richmond, va",226050.0,98.0,245205.0,188.0
4304,23221,"richmond, va",487475.0,31.0,652035.0,58.0
5346,23005,"ashland, va",420050.0,83.0,459728.0,144.0


In [6]:
##Use geopy library to get the latitude and longitude values of new jobs address
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(new_job_addr)
nja_latitude = location.latitude
nja_longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(new_job_addr, nja_latitude, nja_longitude))
print(location.raw['display_name'])

The geograpical coordinates of 7100 Forest Ave 23226 are 37.60536995, -77.5272805050902.
7100, Forest Avenue, Laurel, Crestview, Henrico County, Virginia, 23226, United States of America


In [7]:
coords = []
 
geolocator = Nominatim(user_agent="my-application")
for zipcode, city in zip(rva_df['postal_code'],rva_df['zip_name'] ):
    location = geolocator.geocode({"postalcode": zipcode, "state": 'VA'})
    latitude2 = location.latitude
    longitude2 = location.longitude
    ##print(location)
    miles = distance.distance((nja_latitude,nja_longitude),(latitude2,longitude2)).mi
    #print(round(miles))
    coords.append((
        zipcode,
        latitude2,
        longitude2,
        miles,
        location.raw['display_name']))

GeocoderUnavailable: Service not available

In [ ]:
# dconvert the list into a new DataFrame
coords_df = pd.DataFrame(coords)
#define the column names
coords_df.columns = ['ZipCode','Latitude', 'Longitude', 'Distance','display_name']

print(coords_df.shape)
coords_df.head()

In [ ]:
mylist = coords_df['display_name'].to_list()
cleaner = []
for row in mylist:
    x = row.split(',')
    cleaner.append((x.pop(0)))
coords_df['display_name'] = pd.Series(cleaner)
coords_df.head()

In [ ]:
# create map
map_zip = folium.Map(location=[nja_latitude, nja_longitude], zoom_start=12)

label = '7100 Forest Ave 23226'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [nja_latitude, nja_longitude],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#FDBB84',
    fill_opacity=0.7).add_to(map_zip)  

 
for lat, lng, zipcode, dispname in zip(coords_df['Latitude'], coords_df['Longitude'],coords_df['ZipCode'], coords_df['display_name']):
    label = '{}, {}'.format(dispname, zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_zip)        
map_zip

In [ ]:
##HIDE ME

{
    "tags": [
        "hide_input",
    ]
}
CLIENT_ID = 'CK44XVF0XHD341ERVYWMTKQFUZGM2METCHMV3WNQL1AOCARP' # your Foursquare ID
CLIENT_SECRET = 'M4FVY0XDZLVOBBUZRMSESEE54Z0NH43I1IT5NSAA5ZVDYHB1' # your Foursquare Secret

In [2]:
##get the area info for goods and services
## Foursquare API required inputs
VERSION = '20180604'
LIMIT = 100
radius = 5000
venue_list = []

##loop thru and get info from 4square
for lat, lng, zipcode, dispname in zip(coords_df['Latitude'], coords_df['Longitude'],coords_df['ZipCode'], coords_df['display_name']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()

    items = results['response']['groups'][0]['items']
    ##print(items)
    ##break
    for venue in items:
        try:
            venue_list.append(( 
                zipcode,
                dispname, 
                venue['venue']['name'], 
                venue['venue']['location']['address'],
                venue['venue']['categories'][0]['name']))
        except:
            continue

NameError: name 'coords_df' is not defined

In [ ]:
# dconvert the venue list into a new DataFrame
venues_df = pd.DataFrame(venue_list)
#define the column names
venues_df.columns = ['ZipCode','Display_Name', 'VenueName', 'VenueAddress', 'Category']

print(venues_df.shape)
venues_df.head()

In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['Category'].unique())))

In [ ]:
rva_gs = ['Grocery Store', 'Supermarket','Drugstore', 'Pharmacy', 'Market', 'Coffee Shop', 'Wine Shop','Gym / Fitness Center','Salon / Barbershop','Gym', 'Sports Club','Movie Theater','Shopping Plaza','Shopping Mall', 'Farmers Market','Art Gallery', 'Food & Drink Shop', 'Performing Arts Venue','Museum', 'Theater','Butcher', 'Art Museum','Dog Run','Social Club','State / Provincial Park', 'Bank', 'Lake']
rva_ven = venues_df.loc[venues_df['Category'].isin(rva_gs)]
rva_ven

In [ ]:
rva_onehot = pd.get_dummies(rva_ven[['Category']], prefix="", prefix_sep="")

# add borough and neighborhood column back to dataframe
 
rva_onehot['ZipCode'] = rva_ven['ZipCode'] 
rva_onehot['Display_Name'] = rva_ven['Display_Name']

#  borough and neighborhood column to the first column
fixed_columns = list(rva_onehot.columns[-2:]) + list(rva_onehot.columns[:-2])
rva_onehot = rva_onehot[fixed_columns]

print(rva_onehot.shape)
rva_onehot.head()

rva_grouped = rva_onehot.groupby(['ZipCode']).sum().reset_index()

print(rva_grouped.shape)
rva_grouped.head()